In [1]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# # Importing the dataset
#data = pd.read_csv('Dataset/Internal/ConsolidateOutput.csv')
data = pd.read_csv('internal50.csv')
data= data.drop('NO.',axis=1)
data.columns

Index(['ma_internals', 'ph100_internals', 'be10105_internals',
       'be110_internals', 'be103_internals', 'ec100_internals',
       'ph110_internals', 'ec110_internals', 'cs110_internals',
       'CY100_internals', 'BE100_internals', 'EE100_internals',
       'CY110_internals', 'EE110_internals', 'MA102_internals',
       'BE102_internals', 'CS100_internals', 'CS120_internals',
       'CS202_internas', 'IT202_internals', 'IT204_internals',
       'HS200_internals', 'MA202_internals', 'CS208_internals',
       'IT232_internals', 'IT234_internals', 'CS305_internals',
       'IT301_internals', 'IT303_internals', 'IT305_internals',
       'IT307_internals', 'IT341_internas', 'IT331_internals',
       'IT333_internals', 'IT367_internals', 'IT363_internals',
       'HS300_internals', 'CS304_internals', 'IT304_internals',
       'IT306_internals', 'IT364_internals', 'IT302_internals',
       'IT332_internals', 'IT334_internals', 'IT352_internals',
       'IT366_internals'],
      dtype='obj

In [3]:
# Preparing the training set and the test set
training_set = pd.read_csv('all_train_int50.csv')
training_set= training_set.drop('NO.',axis=1)
training_set = np.array(training_set, dtype = 'int')#user_id,movie_id,ratings
test_set = pd.read_csv('all_test_int50.csv')
test_set= test_set.drop('NO.',axis=1)
test_set = np.array(test_set, dtype = 'int')
test_set.shape
#training_set.shape

(11, 46)

In [4]:
test_set.shape
#training_set.shape

(11, 46)

In [5]:
#nb_students = int(max(max(training_set[:,0]), max(test_set[:,0])))
#nb_courses = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_students = max(training_set.shape[0], test_set.shape[0])-1
nb_courses = max(training_set.shape[1], test_set.shape[1])
nb_students
#nb_courses

42

In [6]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [7]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_courses, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_courses)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x.view(x.size(0), -1) 
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [8]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_students):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            
            train_loss += np.sqrt(loss.item()*mean_corrector)

            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 2.0111712890123832e+18
epoch: 2 loss: 2.0111712890123832e+18
epoch: 3 loss: 2.0111712890123832e+18
epoch: 4 loss: 2.0111712890123832e+18
epoch: 5 loss: 2.0111712890123832e+18
epoch: 6 loss: 2.0111712890123832e+18
epoch: 7 loss: 2.0111712890123832e+18
epoch: 8 loss: 2.0111712890123832e+18
epoch: 9 loss: 2.0111712890123832e+18
epoch: 10 loss: 2.0111712890123832e+18
epoch: 11 loss: 2.0111712890123832e+18
epoch: 12 loss: 2.0111712890123832e+18
epoch: 13 loss: 2.0111712890123832e+18
epoch: 14 loss: 2.0111712890123832e+18
epoch: 15 loss: 2.0111712890123832e+18
epoch: 16 loss: 2.0111712890123832e+18
epoch: 17 loss: 2.0111712890123832e+18
epoch: 18 loss: 2.0111712890123832e+18
epoch: 19 loss: 2.0111712890123832e+18
epoch: 20 loss: 2.0111712890123832e+18
epoch: 21 loss: 2.0111712890123832e+18
epoch: 22 loss: 2.0111712890123832e+18
epoch: 23 loss: 2.0111712890123832e+18
epoch: 24 loss: 2.0111712890123832e+18
epoch: 25 loss: 2.0111712890123832e+18
epoch: 26 loss: 2.0111712890123832

In [9]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_students):
    input = Variable(training_set[id_user]).unsqueeze(0)
    #target = Variable(test_set[id_user]).unsqueeze(0)
for id_user in range(test_set.shape[0]):
    target = Variable(test_set[id_user]).unsqueeze(0)


    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_courses/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)

        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 1.966429543785546e+18


In [10]:
dat = pd.read_csv('all_test.csv')
dat= dat.drop('NO.',axis=1)
columns=dat.columns
c=columns.tolist()
d=pd.DataFrame(c)
d

,0
0,MA101
1,PH100
2,BE10105
3,BE110
4,BE103
5,EC100
6,PH110
7,EC110
8,CS110
9,CY100


In [11]:
student_id = 2

student_grade = training_set.data.numpy()[student_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[student_id, :].reshape(-1,1)
 
user_input = Variable(training_set[student_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
for i in range(0,46):
    if(predicted[i]<0 and predicted[i]>(-100)):
        predicted[i] = predicted[i]/(-2)
    if(predicted[i]<(-100)):
        predicted[i] = predicted[i]/(-10)
    predicted[i] = round(int(predicted[i]))
# Join all info in one dataset
result_array = np.hstack([d, user_target, predicted])
result_array = result_array[result_array[:, 1] > 0]
result_df = pd.DataFrame(data=result_array, columns=['NO.', 'Target Rating',2 ])

In [12]:
result_df.head(44)

,NO.,Target Rating,2
0,MA101,45,35
1,PH100,46,37
2,BE10105,42,43
3,BE110,47,39
4,BE103,41,40
5,EC100,50,39
6,PH110,48,46
7,EC110,49,43
8,CS110,45,44
9,CY100,42,45


In [13]:
result_df=result_df.drop(["Target Rating"],axis=1)
#result_df=result_df.drop(,axis=0)

In [14]:
result_df = result_df.transpose()
result_df

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
NO.,MA101,PH100,BE10105,BE110,BE103,EC100,PH110,EC110,CS110,CY100,...,IT367,HS300,CS304,IT304,IT306,IT364,IT302,IT332,IT334,IT352
2,35,37,43,39,40,39,46,43,44,45,...,43,38,38,39,38,42,40,41,41,37


In [15]:
result_df.to_csv (r'newtest.csv', index = True, header=False)